In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.1 MB 9.1 MB/s 
     |████████████████████████████████| 3.3 MB 50.4 MB/s 
     |████████████████████████████████| 895 kB 53.8 MB/s 
     |████████████████████████████████| 596 kB 48.2 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 1.2 MB 10.4 MB/s 


In [2]:
# Mount Google drive to upload datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The path to the data on my drive
D = '/content/drive/My Drive/W266_Project_Data/pmi_data'

In [42]:
import pandas as pd
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [5]:
# Load the gold-standard target translation datasets
hi_target = pd.read_csv(D+"/predicted_text/MBart/dev.hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ml_target = pd.read_csv(D+"/predicted_text/MBart/dev.ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ta_target = pd.read_csv(D+"/predicted_text/MBart/dev.ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])

In [6]:
print(len(hi_target))
print(len(ml_target))
print(len(ta_target))

1000
1000
1000


In [7]:
hi_target.head()

,sentence_1
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [8]:
# Load the IndicTrans translated files into their respective language datasets
hi_indictrans = pd.read_csv(D+"/predicted_text/IndicTrans/dev/indic_trans_hi_dev.csv")
hi_indictrans.drop(columns={'Unnamed: 0'}, inplace=True)
hi_indictrans.rename({'0': 'sentence_2'}, axis=1, inplace=True)
ml_indictrans = pd.read_csv(D+"/predicted_text/IndicTrans/dev/indic_trans_ml_dev.csv")
ml_indictrans.drop(columns={'Unnamed: 0'}, inplace=True)
ml_indictrans.rename({'0': 'sentence_2'}, axis=1, inplace=True)
ta_indictrans = pd.read_csv(D+"/predicted_text/IndicTrans/dev/indic_trans_ta_dev.csv")
ta_indictrans.drop(columns={'Unnamed: 0'}, inplace=True)
ta_indictrans.rename({'0': 'sentence_2'}, axis=1, inplace=True)

In [9]:
hi_indictrans

,sentence_2
0,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...
1,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो..."
...,...
995,म्यांमार के संघ निर्वाचन आयोग और
996,प्रधानमंत्री ने कहा कि आजादी के बाद से बहुत कु...
997,"उन्होंने उनके साथ जन भागीदारी, सूचना प्रवाह, स..."
998,हाल ही में ग्राम स्वराज अभियान और आयुष्मान भार...


In [10]:
print(len(hi_indictrans))
print(len(ml_indictrans))
print(len(ta_indictrans))

1000
1000
1000


In [11]:
# Create Hindi IndicTrans test set
hi_indictrans_test = pd.concat([hi_target['sentence_1'], hi_indictrans['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
hi_indictrans_test.insert(0, 'language_task', 'translate English to Hindi')
hi_indictrans_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो..."


In [12]:
# Create Malayalam IndicTrans test set
ml_indictrans_test = pd.concat([ml_target['sentence_1'], ml_indictrans['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ml_indictrans_test.insert(0, 'language_task', 'translate English to Malayalam')

In [13]:
# View first few rows of Malayalam IndicTrans test set
ml_indictrans_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...
1,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നതി...
2,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...
3,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...
4,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷു..."


In [14]:
# Load the first few rows of the Tamil Bart test set
ta_indictrans_test = pd.concat([ta_target['sentence_1'], ta_indictrans['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ta_indictrans_test.insert(0, 'language_task', 'translate English to Tamil')

In [15]:
# View the first few rows of the Tamil translation
ta_indictrans_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Tamil,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,கோடிக்கணக்கான மக்களின் மனதிலும் இதயத்திலும் வி...
1,translate English to Tamil,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,பீஜப்பூரில் இன்று இந்த நிகழ்ச்சியை நடத்துவதன் ...
2,translate English to Tamil,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,இந்தப் பணியை 2022-க்குள் முடிக்க இலக்கு நிர்ணய...
3,translate English to Tamil,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,தெளிவான இலக்குகள் மற்றும் காலக்கெடுவுடன் பல்வே...
4,translate English to Tamil,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","இந்த சூழலில், ஜன் தன் கணக்குகளைத் தொடங்குதல், ..."


In [21]:
class SiameseNet(torch.nn.Module):
    def __init__(self):
      super(SiameseNet, self).__init__()

      # Set up a fully-connected layer to apply to each incoming sentence vector
      # Use a dense layer (nn.Linear) to reduce the dimensionality of the input embeddings
      # use a relu activation function
      # Randomly drop out
      # Set 1 as the dimensionality of the final output since we want to predict the logits corresponding to a binary label
      # Initialize the feed forward layer
      self.fc = torch.nn.Sequential(
            torch.nn.Linear(768, 128),
            torch.nn.InstanceNorm1d(128),
            torch.nn.ReLU(inplace=True))
      
      self.relu = torch.nn.ReLU(inplace=True)
      self.batch2 = torch.nn.InstanceNorm1d(384)
      self.ffn = torch.nn.Linear(384, 128)
      self.batch3 = torch.nn.InstanceNorm1d(128)
      self.layer_out = torch.nn.Linear(128,1)

      # Add drop out
      self.drop = torch.nn.Dropout(0.2)

    # left and right represent columns in our data
    def forward(self, left, right):

      # Get the sentence embeddings 
      u = left
      v = right

      # Reduce the dimensionality
      u = self.fc(u)
      v = self.fc(v)

      # Get the absolute value of the difference of the vectors
      uv = torch.sub(u, v)
      uv_abs = torch.abs(uv)

      # concatenate u, v, and the absolute value of the element-wise difference between the embeddings
      x = torch.cat([u, v, uv_abs], dim=-1)

      # Normalize the concatenated batch
      x = self.batch2(x)

      # Use the relu activation function
      x = self.relu(x)

      # Pass the concatenated layers through a dense layer to reduce the dimensionality from 2304 to 128
      x = self.ffn(x)
      # Normalize the concatenated layers
      x = self.batch3(x)
      # Pass through relu activation
      x = self.relu(x)
      # Implement drop out to avoid overfitting
      x = self.drop(x)
      # Pass the output through a final dense layer
      output = self.layer_out(x)
      # Reshape the output to have the same dimension as the target labels
      output = output[:,-1,:]

      # Concatenate the 
      return output

In [18]:
# Set up the device to run on GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [19]:
# Create an instance of the tokenizer and model using Indic Bert
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', keep_accents=True)
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def sentence_embeddings(device, tokenizer, model, sentences):
  """Create contextualized sentence embeddings
  
  Parameters:
  tokenizer: transformers tokenizer for creating word tokens
  model:
  sentences: sentences to generate sentence embeddings for
  """

  # Tokenize sentences and return PyTorch tensors
  encoded_input = tokenizer(sentences, padding=True, truncation=True, 
                            max_length=128, return_tensors='pt')
  encoded_input = encoded_input.to(device)
  
  # Pass the tokenized input to the model 
  with torch.no_grad():
    model.to(device)
    model_output = model(**encoded_input) 
  
  # Perform mean pooling to get total sentence embeddings
  sentence_embeddings = mean_pooling(model_output, 
                                     encoded_input['attention_mask'])
  
  return sentence_embeddings

In [22]:
# Load the Siamese Neural Net from the pre-trained weights
net = SiameseNet().to(device)
# Load the weights and biases from the last round of training
net.load_state_dict(torch.load("/content/drive/My Drive/W266_Project_Data/models/siamese_model/three_dense_w_punjabi_model.pt"))

<All keys matched successfully>

In [23]:
# Create a dataset class, similarly to the one used for training the Siamese Neural Network dataset - the difference is that it won't create labels
class TranslationEvalDataset(Dataset): 
  def __init__(self,training_df, device, tokenizer, model):
    self.training_df=training_df
    self.sent1 = self.training_df['sentence_1'].tolist()
    self.sent2 = self.training_df['sentence_2'].tolist()
    self.language_task = self.training_df['language_task'].tolist()
    self.device = device
    self.tokenizer = tokenizer
    self.model = model

  def __getitem__(self, index):
    sent1 = self.sent1[index]
    sent2 = self.sent2[index]
    task = self.language_task[index]

    # Get the sentence embeddings for sentences 1 and 2 for each row
    sent1_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent1)
    sent2_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent2)
    return sent1_embeddings, sent2_embeddings, task

  def __len__(self):
    return len(self.training_df)

In [24]:
# Concatenate all of the bart test sets into one test set
final_test = pd.concat([hi_indictrans_test, ml_indictrans_test, ta_indictrans_test]).reset_index()
final_test.drop(columns={'index'}, inplace=True)

In [25]:
final_test[1000:2000]

,language_task,sentence_1,sentence_2
1000,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...
1001,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നതി...
1002,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...
1003,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...
1004,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷു..."
...,...,...,...
1995,translate English to Malayalam,മ്യാന്‍മാറിലെ യൂണിയന്‍ തിരഞ്ഞെടുപ്പ് കമ്മിഷന്‍ 7.,മ്യാൻമർ യൂണിയൻ ഇലക്ഷൻ കമ്മീഷൻ
1996,translate English to Malayalam,സ്വാതന്ത്ര്യാനന്തരം പല നേട്ടങ്ങളും ഉണ്ടാക്കാന്...,സ്വാതന്ത്ര്യം ലഭിച്ചതിനുശേഷം നിരവധി നേട്ടങ്ങൾ ...
1997,translate English to Malayalam,"ജന്‍ ഭാഗീദാരി, വിവരം പങ്കിടല്‍, വിഭവങ്ങള്‍ പരമ...","ജനപങ്കാളിത്തം, വിവരപ്രവാഹം, വിഭവങ്ങളുടെ പരമാവധ..."
1998,translate English to Malayalam,"ഗ്രാമ സ്വരാജ് അഭിയാന്‍, ആയുഷ്മാന്‍ ഭാരത് തുടങ്...","ഗ്രാമ സ്വരാജ് അഭിയാൻ, ആയുഷ്മാൻ ഭാരത് തുടങ്ങിയ ..."


In [26]:
final_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो..."


In [27]:
final_test.tail()

,language_task,sentence_1,sentence_2
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,மியான்மர் ஒன்றிய தேர்தல் ஆணையம் மற்றும்
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,நாடு சுதந்திரம் அடைந்ததிலிருந்து பல சாதனைகள் ப...
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...","மக்கள் பங்கேற்பு, தகவல் பரிமாற்றம், ஆதாரங்களை ..."
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...",அண்மையில் மேற்கொள்ளப்பட்ட கிராம சுயாட்சி இயக்க...
2999,translate English to Tamil,இந்தியாவின் 130 லட்சம் மக்கள் சார்பில் உங்கள் ...,இந்தியாவின் 130 கோடி மக்களின் சார்பாக உங்கள் அ...


In [28]:
# Create an instance of the test set with all test translations
indictrans_test = TranslationEvalDataset(final_test, device, tokenizer, model)

In [29]:
# Create an instance of the test data loader
indictrans_test_loader = DataLoader(indictrans_test, batch_size=16)

In [30]:
# Track the language_tasks and sentences  being evaluated
y_language_task = final_test.language_task.tolist()
y_sent1 = final_test.sentence_1.tolist()
y_sent2 = final_test.sentence_2.tolist()
probability_of_paraphrase = []

with torch.no_grad():

  # Place the model into evaluation mode
    net.eval()

    # Loop over the test set
    for i, data in enumerate(indictrans_test_loader, 0):

      # Extract the test set components
      sent1_test, sent2_test, task_test = data

      # Send the test data to GPU
      sent1_test, sent2_test = sent1_test.to(device), sent2_test.to(device)

      # Get the test set predictions
      test_pred = net(sent1_test, sent2_test)
            
      # Get the predicted test labels
      y_test_pred = torch.sigmoid(test_pred).float()

      # Add the test labels to their respective list
      probability_of_paraphrase.extend(y_test_pred)

In [31]:
# Create a dataframe of the tasks, sentences, and probabilities of the second sentence (the translated one) being a paraphrase
y_probs = [x.item() for x in probability_of_paraphrase]
indictrans_performance = pd.DataFrame(list(zip(y_language_task, y_sent1, y_sent2, y_probs)),columns =['language_task', 'target_sentence', 'indictrans_translation', 'probability_of_paraphrase'])

In [32]:
# Check the first few rows
indictrans_performance.head()

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.989960
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.989784
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990179
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.987861


In [33]:
# Sort the dataframe in order of lowest predicted probability and see where IndicTrans returns the lowest scores
indictrans_performance.sort_values("probability_of_paraphrase")

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase
2925,translate English to Tamil,சிவன் – பார்வதி ஆகிய இருவரின் சேர்க்கை என்பது ...,இமயமலையும் பெருங்கடலையும் இணைப்பதே சிவன்-பார்வ...,0.011562
2670,translate English to Tamil,"அரசியல் களத்தைத் தவிர, தமிழ்த்திரைப்பட உலகில் ...",தமிழ் திரையுலகில் மட்டுமல்லாமல் அரசியலிலும் தன...,0.012262
1235,translate English to Malayalam,ഇത്തരം പ്രാദേശിക പരിശ്രമങ്ങള്‍ക്ക് അന്തര്‍ദ്ദേ...,ശക്തമായ അന്താരാഷ്ട്ര പങ്കാളിത്തത്തിലൂടെ ഇത്തരം...,0.013479
1494,translate English to Malayalam,രാജ്യങ്ങള്‍ക്കിടയിലുള്ള വാണിജ്യം ഫലപ്രദമാക്കുന...,സാമ്പത്തിക വികസനവും പരിസ്ഥിതി സംരക്ഷണവും തമ്മി...,0.013571
2384,translate English to Tamil,நான் இந்த விஷயத்தில் அதிக விழிப்புணர்வு ஏற்பட ...,இந்த பிரச்சனை குறித்து நான் அதிகம் வலியுறுத்தவ...,0.013753
...,...,...,...,...
978,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार लोगों की आकांक्षा...,प्रधानमंत्री ने कहा कि सरकार लोगों की आकांक्षा...,0.991897
403,translate English to Hindi,यह विभिन्न क्षेत्रों में हमारी साझेदारी में बह...,यह विभिन्न क्षेत्रों में हमारी साझेदारी में बड...,0.991987
36,translate English to Hindi,"वे भारतीय प्रौद्योगिकी संस्थान, बम्बई के 56वें...","वह भारतीय प्रौद्योगिकी संस्थान, बॉम्बे के 56वे...",0.992162
859,translate English to Hindi,मैं वाइब्रेंट गुजरात समिट में आप सभी का स्वागत...,मैं आप सभी का वाइब्रेंट गुजरात समिट में स्वागत...,0.992416


In [34]:
# Check the number of rows with a predicted probability <0.5 (meaning that they would be assigned a label of 0 - not a paraphrase)
len(indictrans_performance[indictrans_performance.probability_of_paraphrase < 0.5])

1037

In [35]:
# Check the number of translations of each language where the translation is not counted as a paraphrase
hi_errors = len(indictrans_performance[(indictrans_performance.probability_of_paraphrase < 0.5) & (indictrans_performance.language_task == 'translate English to Hindi')])
ml_errors = len(indictrans_performance[(indictrans_performance.probability_of_paraphrase < 0.5) & (indictrans_performance.language_task == 'translate English to Malayalam')])
ta_errors = len(indictrans_performance[(indictrans_performance.probability_of_paraphrase < 0.5) & (indictrans_performance.language_task == 'translate English to Tamil')])
print(hi_errors)
print(ml_errors)
print(ta_errors)

41
447
549


In [36]:
# The comparatively high error rate for Malayalam and Tamil reflect their comparatively lower Sacre Bleu scores
hi_error_rate = (hi_errors/1000)
ml_error_rate = (ml_errors/1000)
ta_error_rate = (ta_errors/1000)

In [37]:
# Hindi performs fairly well
print("{0:.2%}".format(hi_error_rate))

4.10%


In [38]:
# Malayalam error rate is high
print("{0:.2%}".format(ml_error_rate))

44.70%


In [39]:
# The Tamil error rate is even higher than Malayalam
print("{0:.2%}".format(ta_error_rate))

54.90%


In [40]:
# Get the sentence embeddings for a pair of "dissimilar sentences"
sent1_embed = sentence_embeddings(device, tokenizer, model, indictrans_performance['target_sentence'][2925])
sent2_embed = sentence_embeddings(device, tokenizer, model, indictrans_performance['indictrans_translation'][2925])

In [43]:
# Find the cosine similarity score between these pair of "dissimilar sentences"
cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

print(cosine_similarity_value)

tensor(0.8800, device='cuda:0')


In [44]:
# Try for another pair of sentences, this time, for those with a high similarity score
sent3_embed = sentence_embeddings(device, tokenizer, model, indictrans_performance['target_sentence'][1])
sent3_embed = sentence_embeddings(device, tokenizer, model, indictrans_performance['indictrans_translation'][1])

In [45]:
# The sentences are the same
F.cosine_similarity(sent3_embed.squeeze(0), sent3_embed.squeeze(0), dim=0)

tensor(1., device='cuda:0')

In [46]:
# Get the sentence embeddings and cosine similarity scores between each pair of sentences in the dataset
# This is to check if low scoring sentences have exact similarity scores or if they are of low quality
def check_cosine_similarity(df, device, tokenizer, model):

  # Create an empty list to check cosine similarity scores
  cosine_scores = []

  for index, rows in df.iterrows():
    # Get the sentence embeddings of each pair of target sentences
    sent1_embed = sentence_embeddings(device, tokenizer, model, rows['target_sentence'])
    sent2_embed = sentence_embeddings(device, tokenizer, model, rows['indictrans_translation'])

    # Reshape the embeddings to be of a single dimension 
    # Get the cosine similarity between the two pairs of sentence embeddings
    cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

    # Add the cosine similarity score to the list of scores
    cosine_scores.append(cosine_similarity_value.item())

  return cosine_scores

In [47]:
translation_scores = check_cosine_similarity(indictrans_performance, device, tokenizer, model)

In [48]:
# Add the translation similarity scores to the indictrans performance dataframe
indictrans_performance['cosine_similarity_scores'] = translation_scores

In [49]:
# Sentences with low cosine similarity also have low probability of paraphrase
indictrans_performance.sort_values('cosine_similarity_scores')

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase,cosine_similarity_scores
1222,translate English to Malayalam,"‘ഏറ്റവും കുറഞ്ഞ ഗവണ്‍മെന്റ്, പരമാവധി ഭരണ നിര്‍...","'മിനിമം ഗവൺമെന്റ്, മാക്സിമം ഗവേണൻസ്', 'മിനിമം ...",0.020203,0.393084
1716,translate English to Malayalam,ഒരു വമ്പിച്ച പൊതുയോഗത്തിൽ പി എം ജെ എ വൈ ഉദ്‌ഘാ...,പി. എം. ജെ. എ. വൈ.,0.044626,0.426306
357,translate English to Hindi,भारतीय पक्षकार,भारतीय पक्ष द्वारा आदान-प्रदान किया गया,0.107770,0.476889
37,translate English to Hindi,पृष्ठ भूमिः,पृष्ठभूमि:,0.037115,0.488031
39,translate English to Hindi,आप सबका धन्यवाद!,आपका बहुत-बहुत धन्यवाद।,0.070636,0.540804
...,...,...,...,...,...
394,translate English to Hindi,हमारे प्रमुख राष्ट्रीय प्रतीकों ने बौद्ध धर्म ...,हमारे प्रमुख राष्ट्रीय प्रतीकों ने बौद्ध धर्म ...,0.989735,1.000000
118,translate English to Hindi,यह एक ऐतिहासिक उपलब्धि है।,यह एक ऐतिहासिक उपलब्धि है।,0.990184,1.000000
821,translate English to Hindi,प्रधानमंत्री ने पुल के निर्माण में शामिल सभी ल...,प्रधानमंत्री ने पुल के निर्माण में शामिल सभी ल...,0.991094,1.000000
161,translate English to Hindi,प्रधानमंत्री ने नवसारी में निराली कैंसर अस्पता...,प्रधानमंत्री ने नवसारी में निराली कैंसर अस्पता...,0.986947,1.000000


In [50]:
# High similarity scores have a high probability of paraphrase
indictrans_performance.sort_values('cosine_similarity_scores', ascending=False)

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase,cosine_similarity_scores
821,translate English to Hindi,प्रधानमंत्री ने पुल के निर्माण में शामिल सभी ल...,प्रधानमंत्री ने पुल के निर्माण में शामिल सभी ल...,0.991094,1.000000
665,translate English to Hindi,उत्तर प्रदेश में फूलपुर-हल्दिया गैस पाइपलाइन प...,उत्तर प्रदेश में फूलपुर-हल्दिया गैस पाइपलाइन प...,0.986729,1.000000
1821,translate English to Malayalam,പാലത്തിന്റെ നിർമ്മാണത്തിൽ പങ്കാളികളായ എല്ലാവരെ...,പാലത്തിന്റെ നിർമ്മാണത്തിൽ പങ്കാളികളായ എല്ലാവരെ...,0.988308,1.000000
140,translate English to Hindi,"इनमें सरकारी सिविल अस्पताल, एक दुग्ध प्रसंस्कर...","इनमें सरकारी सिविल अस्पताल, एक दुग्ध प्रसंस्कर...",0.990638,1.000000
297,translate English to Hindi,भ्रष्टाचार के लिए कोई जगह नहीं है।,भ्रष्टाचार के लिए कोई जगह नहीं है।,0.990275,1.000000
...,...,...,...,...,...
39,translate English to Hindi,आप सबका धन्यवाद!,आपका बहुत-बहुत धन्यवाद।,0.070636,0.540804
37,translate English to Hindi,पृष्ठ भूमिः,पृष्ठभूमि:,0.037115,0.488031
357,translate English to Hindi,भारतीय पक्षकार,भारतीय पक्ष द्वारा आदान-प्रदान किया गया,0.107770,0.476889
1716,translate English to Malayalam,ഒരു വമ്പിച്ച പൊതുയോഗത്തിൽ പി എം ജെ എ വൈ ഉദ്‌ഘാ...,പി. എം. ജെ. എ. വൈ.,0.044626,0.426306


In [51]:
# Find the range in cosine similarity scores for sentences not considered paraphrases of each other
np = indictrans_performance[indictrans_performance.probability_of_paraphrase < 0.5].reset_index()
np.drop(columns={'index'}, inplace=True)

In [52]:
# Find the min and max of the cosine similarity scores
print(min(np.cosine_similarity_scores))
# There are some sentences labeled as not paraphrases of each other that have a high degree of cosine similarity
print(max(np.cosine_similarity_scores))

0.3930836021900177
0.9825106859207153


In [53]:
# Do the same for sentences considered paraphrases of each other
p = indictrans_performance[indictrans_performance.probability_of_paraphrase >= 0.5]
print(min(p.cosine_similarity_scores))
print(max(p.cosine_similarity_scores))

0.5508232712745667
1.0


In [54]:
# Look at the pair of sentences with the highest degree of cosine similarity
np[np['cosine_similarity_scores'] > 0.5]

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase,cosine_similarity_scores
1,translate English to Hindi,आप सबका धन्यवाद!,आपका बहुत-बहुत धन्यवाद।,0.070636,0.540804
2,translate English to Hindi,मेरे प्यारे देशवासियो !,"मेरे प्यारे देशवासियों, नमस्कार।",0.046447,0.601346
3,translate English to Hindi,NDRF की टीमें लगाई गईं।,एनडीआरएफ की टीमों को तैनात किया गया है।,0.380256,0.787473
4,translate English to Hindi,एक phone call मैं आपको भी सुनाता हूँ: –,मैं चाहूंगा कि आप ऐसे ही एक फोन कॉल को सुनेंः -,0.075742,0.696464
5,translate English to Hindi,क्या कोई समाचार पत्र इस कार्य के लिए प्रतिदिन ...,"क्या एक समाचार पत्र, हर दिन एक साल के लिए, केव...",0.256926,0.818762
...,...,...,...,...,...
1032,translate English to Tamil,மத்திய அரசின் நிதி உள்ளடக்கம் மற்றும் சமூக பாத...,அனைவரையும் உள்ளடக்கிய நிதி மற்றும் சமூகப் பாது...,0.189554,0.937887
1033,translate English to Tamil,“20 ஓவர் கிரிக்கெட் உலகக் கோப்பையை இந்தியா வென...,பார்வையற்றோருக்கான டி20 உலகக் கோப்பையை இந்தியா...,0.026242,0.892837
1034,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,மியான்மர் ஒன்றிய தேர்தல் ஆணையம் மற்றும்,0.248196,0.760211
1035,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...","மக்கள் பங்கேற்பு, தகவல் பரிமாற்றம், ஆதாரங்களை ...",0.338175,0.961721


In [55]:
# Get the target sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'target_sentence'].iloc[0]

'அடல் புதுமை இயக்கம், திறன் இந்தியா, தொடங்கிடு இந்தியா போன்ற திட்டங்களைப்பற்றியும் அவர் குறிப்பிட்டார்.'

In [56]:
# Get the translation sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'indictrans_translation'].iloc[0]

'அடல் புதுமை இயக்கம், திறன் இந்தியா, தொடங்கிடு இந்தியா ஆகியவற்றையும் அவர் குறிப்பிட்டார்.'

In [58]:
# The English translation of the target sentence with the maximum cosine similarity score - using Google translate to translate to English
english_np_target = 'He also mentioned about projects like Atal Innovation Movement, Skill India, Start India.'
english_np_translation = 'He also mentioned Atal Innovation Movement, Skill India and Start India.'
# As we can see in the examples below, the sentences mean similar things - only 1 term is missing (the concept of projects)
# The language is Tamil
print(english_np_target)
print(english_np_translation)

He also mentioned about projects like Atal Innovation Movement, Skill India, Start India.
He also mentioned Atal Innovation Movement, Skill India and Start India.


In [60]:
# Let's look at the sentence pair with the lowest cosine similarity in the paraphrase dataset
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'target_sentence'].iloc[0])
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'indictrans_translation'].iloc[0])

പശ്ചാത്തലം: 1.
പശ്ചാത്തലംഃ


In [62]:
# Print the English translations of these sentences - obtained using Google translate to back-translate
# These sentences have a low cosine similarity due to the translation missing the number 1, but the meaning is the same
# The sentences are Malayalam
english_p_target = 'Background: 1.'
english_p_translation = 'Background:'

In [63]:
# Write the bart performance file to a csv 
indictrans_performance.to_csv(D+"/predicted_text/siamese_evaluations_of_translations/indictrans_siamese_paraphrase_eval.csv")